In [11]:

from copy import deepcopy

import torch.optim as optim
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from tqdm import tqdm
import time

In [12]:
from senmodel.model.utils import *
from senmodel.metrics.nonlinearity_metrics import *
from senmodel.metrics.edge_finder import *
from senmodel.metrics.train_metrics import *
from senmodel.train.train import *

In [13]:
torch.manual_seed(0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [14]:
class SimpleFCN(nn.Module):
    def __init__(self, input_size=28 * 28, hidden_size=16):
        super(SimpleFCN, self).__init__()
        self.fc0 = nn.Linear(input_size, 10)
        # self.fc1 = nn.Linear(hidden_size, 10)
        self.act = nn.ReLU()

    def forward(self, x):
        x = self.fc0(x)
        return x

In [15]:
# Dataset and Dataloader
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))
])

# Load dataset and split into train/validation sets
dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

train_dataset, val_dataset, test_dataset = random_split(dataset, [0.6, 0.2, 0.2])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [16]:
model = SimpleFCN()
sparse_model = convert_dense_to_sparse_network(model, layers=[model.fc0])

In [ ]:
hyperparams = {
    "num_epochs": 64,
    "metric": AbsGradientEdgeMetric(nn.CrossEntropyLoss()),
    "aggregation_mode": "mean",
    "choose_thresholds": {"fc0": 0.7},
    "threshold": 0.005,
    "min_delta_epoch_replace": 12,
    "window_size": 5,
    "lr": 1e-4,
    "delete_after": 4,    
}

name = ", ".join(
    f"{key}: {value.__class__.__name__ if key == 'metric' else value}"
    for key, value in hyperparams.items()
)

name

"num_epochs: 64, metric: ReversedAbsGradientEdgeMetric, aggregation_mode: mean, choose_thresholds: {'fc0': 1e-06}, threshold: 0.005, min_delta_epoch_replace: 12, window_size: 5, lr: 0.0001, delete_after: 4"

In [18]:
import wandb

wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [ ]:
wandb.finish()

run = wandb.init(
    project="self-expanding-nets",
    name=f"trash",
    config=hyperparams
)


In [ ]:
train_sparse_recursive(sparse_model, train_loader, val_loader, test_loader, hyperparams)
wandb.finish()

100%|██████████| 563/563 [00:06<00:00, 81.60it/s]


Epoch 1/64, Train Loss: 1.6113, Val Loss: 1.1515, Val Accuracy: 0.8066


100%|██████████| 563/563 [00:06<00:00, 86.15it/s]


Epoch 2/64, Train Loss: 0.9379, Val Loss: 0.7969, Val Accuracy: 0.8387


100%|██████████| 563/563 [00:06<00:00, 87.46it/s]


...

Epoch 37/64, Train Loss: 0.2681, Val Loss: 0.2831, Val Accuracy: 0.9220


100%|██████████| 563/563 [00:12<00:00, 46.88it/s]


Epoch 38/64, Train Loss: 0.2659, Val Loss: 0.2817, Val Accuracy: 0.9218


100%|██████████| 563/563 [00:10<00:00, 52.17it/s]


Epoch 39/64, Train Loss: 0.2641, Val Loss: 0.2801, Val Accuracy: 0.9228
